In [16]:
from base_app.helpers import get_name_mappings
from django.db.models import Q
from base_app.models import ConfigItems
from leagues.models import Competition, Season
import pandas as pd


In [6]:
count_league_to_name_mapping = get_name_mappings(source='sa',target='n',source_as_file_names=True,use_with_region_names=True)

In [4]:
df = pd.read_excel("Retry_Items.xlsx")

In [13]:
c = []
for i,row in df.iterrows() :
    search_string = f"{row['region']}|{row['competition']}"
    existing_alt = row['competition_alt']
    found = count_league_to_name_mapping.get(search_string)
    if existing_alt != found :
        df.loc[i,'competition_alt']=found

In [15]:
df.to_excel("Updated_Retry_items.xlsx")

In [19]:
df.head()

,batch_x,confederation,region,competition,season,type,matches,team_counts,batch_y,fixture_count,competition_alt,diff
0,B6,AFC,AFC,AFC Champions League Elite,2022,events,127,46,B1,137.0,AFC Champions League,10.0
1,B6,AFC,AFC,AFC Champions League Two,2019,events,104,37,B1,141.0,AFC Champions League 2,37.0
2,B6,AFC,AFC,AFC Champions League Two,2024/2025,events,73,31,NaN,NaN,AFC Champions League 2,NaN
3,B6,AFC,Australia,A-League Men,2018/2019,events,129,10,B1,140.0,A-League,11.0
4,B6,AFC,Australia,A-League Men,2024/2025,events,176,13,NaN,NaN,A-League,NaN


In [32]:
q = Q()
for i, row in df.iterrows():
    q1 = (
        Q(competition__confederation=row["confederation"], competition__country=row["region"],competition__competition_name=row['competition_alt']) &
        (Q(name=row["season"]) | Q(name_fotmob=row["season"]))
    )
    q |= q1
    p = Season.objects.filter(q1)
    if len(p)!=1 :
        print(i,len(p),row)
        print()
        print("-------")
qs = Season.objects.select_related("competition").filter(q)
len(qs),df.shape

30 2 batch_x                   B6
confederation       CONCACAF
region                Mexico
competition          Liga MX
season             2024/2025
type                  events
matches                   35
team_counts               15
batch_y                  NaN
fixture_count            NaN
competition_alt      Liga MX
diff                     NaN
Name: 30, dtype: object

-------


(253, (252, 12))

In [26]:
Season.objects.filter()[0].__dict__

dict_keys(['_state', 'id', 'competition_id', 'name', 'name_fotmob', 'season_event_url', 'season_shot_url'])